# FP Growth


In [95]:
import pandas as pd
import numpy as np
import itertools
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import sys
import functools
sys.path.append('..')
from utils.utils import flatten, is_subset_of

In [96]:
data_path = '../data/store_data.csv'

## Class HT
### HT data structure
```cpp
struct ht {
  info: (key, counts),
  next: next_addr;
}
```

- The class HT is used to create the header table, which will save the information of frequent item set in desceding order
- And, header table also have the duty to store the address of each frequent item and each frequent item also point to next itself item

### Function
- build_ht : build the header table
- find_until_none : this method is to save the instance into the linked-list chain
- find_key_in_ht : return the address of desired item(class Node) 

In [97]:
class HT :
    def __init__(self):
        self.ht = None
    
    def build_ht(self, data, min_sup):
        if self.ht == None:
            self.ht = []
        keys, counts = np.unique(flatten(data), return_counts=True)
        ht_arr = np.array([(i,j) for i,j in zip(keys, counts)], dtype=[('key', int), ('counts', int)])
        rst = np.flip(np.sort(ht_arr, order='counts'))
        rst = filter(lambda x: x if x[1]>=min_sup else None, rst)
        rst = [ {'info':d, 'next': None}  for d in rst]
        self.ht = list(rst)
    
    def find_until_none(self, key, addr):
        for d in self.ht:
            if d['info'][0] == key and d['next'] != None:
                cur = d['next']
                while cur.next_instance != None:
                    cur = cur.next_instance
                cur.next_instance = addr
                return
            elif d['info'][0] == key and d['next'] == None:
                d['next'] = addr
                return
        raise "No wanted key it is wierd"
    
    def find_key_in_ht(self, key):
        for d in self.ht:
            if d['info'][0] == key:
                return d

## Class Node
The node will store the following info
- numOfChild : number of children
- _key : the key of this object
- _count : the number of times the frequent item match
- next_instance : store the address of the next same key node
- slots : it is a list which contain the tuple, (key, value(address) )
- parent : the address of its parent
### Functions
- insertNode : insert a child node to this node
- findWithKey : find the key whether it is of its children
- contain_with : return bool value indicate whether the key is its child or not

### Special function
It is made to operate the node more directly
- __add__ : which will add the node as its child and increments its numOfChild
- __gt__ : which will return the child given the key and no match item will return None 

In [98]:
class Node :
    def __init__(self, name):
        self.numOfChild = 0
        self._key = name
        self._count = 0
        self.next_instance = None
        self.slots = None
        self.parent = None

    @classmethod
    def DFS_tra(self, cls):
        if cls.children == None:
            return
        for nkeys,value in cls.children.items():
            print(nkeys, value.count)
            Node.DFS_tra(value)
        
    @classmethod
    def DFS_search(self, cls, addr, path, global_path):
        if cls.children == None:
            return
        for nkeys, value in cls.children.items():
            path.append(value)
            if value == addr:
                global_path.append(path.copy())
                del path[-1]
                return
            else:
                Node.DFS_search(value, addr, path, global_path)
            del path[-1]
                
    @property
    def key(self):
        return self._key
    
    @property
    def children(self):
        return self.slots
    
    @property
    def count(self):
        return self._count
    @count.setter
    def count(self, addOne):
        self._count += addOne
    
    def insertNode(self, node):
        if self.slots == None:
            self.slots = {}
        self.slots[node.key] = node
        node.parent = self
    
    def findWithKey(self, key):
        if self.slots == None:
            return None
        for nkey,value in self.slots.items():
            if nkey == key:
                return value
        return None

    def contain_with(self, key):
        if self.slots == None:
            return False
        for nkey,value in self.slots.items():
            if nkey == key:
                return True
        return False

    def generator(self, cur):
        if cur == None:
            return None
        return cur.next_instance
    
    def __add__(self, other):
        self.insertNode(other)
        self.numOfChild += 1
        other.count = 1
        return other

    def __gt__(self, other_key):
        return self.findWithKey(other_key)
        

In [99]:
root = Node('root')
root + Node(55)
(root > 55) + Node(77)
child = (root > 55) > 77
print(child.parent.key)
print(child.parent.parent.key)
c = (root > 55) + Node(56)
print(c.parent.key)

55
root
55


## FPEstimator(HT)
which inherits from class HT so it have the ability to build the header table and  

In [284]:
class FPEstimator(HT):
    ''' @FPEstimator inherented from HT
        @pro:
            1. data
            2. shape
            3. l1label
            4. Encoder
    '''
    def __init__(self, df_data):
        super().__init__()
        self.df_data = df_data
        self.dshape = df_data.shape
        self.le = LabelEncoder()
        self.extract_items(df_data)
        self.root = None
        self.relations = None
        self.fp_n = None
        self.fp_n_encoded = None
        
    @property
    def data(self):
        return self.df_data
    @property
    def shape(self):
        return self.dshape
    
    @property
    def l1label(self):
        return list(zip(self.itemset, self.l1_count))
    @property
    def Encoder(self):
        return self.le     
    
    def extract_items(self, data):
        self.raw_items = []
        for index, row in data.iterrows():
            row = pd.Series(row).dropna().values
            self.raw_items.append(np.array(row).tolist())
        self.itemset, self.l1_count = np.unique(flatten(self.raw_items), return_counts=True)
        self.le.fit(self.itemset)
        self.encoded_data = [ self.le.transform(d) for d in self.raw_items ]
    
    def order_cmp(self,x,y):
        return self.htl.index(x) - self.htl.index(y)
    
    def build_tree(self, min_sup, print_out=False):
        if self.root == None:
            self.root = Node('r')
        self.build_ht(self.encoded_data, min_sup)
        cur = None
        self.rearr_data = []
        htk = { d['info'][0] for d in self.ht }
        self.htl = [ d['info'][0] for d in self.ht ]
        for index, row in enumerate(self.encoded_data):
            l = list(set(row).intersection(htk))
            l = sorted(l,key=functools.cmp_to_key(self.order_cmp))
            self.rearr_data.append(l)
        for index, row in enumerate(self.rearr_data):
            cur = self.root
            for d in row:
                if print_out:
                    print(d, end=' ')
                if not cur.contain_with(d):
                    cur = cur + Node(d)
                    self.find_until_none(d, cur)
                else:
                    cur = cur.findWithKey(d)
                    cur.count = 1
        self.min_sup = min_sup
        self.relations = None

    def combination_with_pattern(self, ls, bit, key, value, global_ls):
        if bit:
            ls.append((key, value.count))
        if value.children == None:
            if len(ls) != 0:
                global_ls.append(ls.copy())
            return
        for dkey, dvalue in value.children.items():
            self.combination_with_pattern(ls, 0, dkey, dvalue, global_ls)
            self.combination_with_pattern(ls, 1, dkey, dvalue, global_ls)
            ls.pop()

    def gen_list(self, print_cpb=False, print_relation=False):
        if self.relations == None:
            self.relations = {}
        if self.min_sup == None:
            print("You need to execute build_tree first")
        global_path = []
        with tqdm(total = len(self.ht)+1) as pbar:
            for i in range(1, len(self.ht) + 1):
                global_path = []
                global_ls = []
                cur = self.ht[-i]['next']
                #print(f"Find {cur.key}  {self.le.inverse_transform([cur.key])[0]}")
                self.relations[cur.key] = global_ls
                while cur != None:   
                    path = []
                    Node.DFS_search(self.root, cur, path, global_path)
                    cur = cur.next_instance
                if print_cpb:
                    for ls in global_path:
                        for item in ls:
                            print(f"({item.key} {item.count})", end=' ')
                        print('')
                    print()
                # The section to generate the relation
                temp = {}
                for ls in global_path:
                    for item in ls[:-1]:
                        if item.key in temp:
                            temp[item.key] += ls[-1].count
                        else:
                            temp[item.key] = ls[-1].count
                # Generate the conditional pattern base
                root = Node('r')
                t = None
                for ls in global_path:
                    r = root
                    for item in ls[:-1]:
                        if temp[item.key] >= self.min_sup and not r.contain_with(item.key):
                            t = Node(item.key)
                            t.count = (ls[-1].count -1)
                            r = r + t
                        elif temp[item.key] >= self.min_sup and r.contain_with(item.key):
                            r = (r > item.key)
                            r.count = (ls[-1].count)
                # Target and pattern base do the combination with the comditional pattern
                if root.children == None:
                    pbar.update(1)
                    continue
                for key, value in root.children.items():
                    ls = []
                    self.combination_with_pattern(ls, 0, key, value, global_ls)
                    self.combination_with_pattern(ls, 1, key, value, global_ls)
                if print_relation:
                    for relation in global_ls:
                        for item in relation:
                            print(f"({self.le.inverse_transform([item[0]])[0]} {item[1]})", end=' ')
                        print()
                    print()
                pbar.update(1)

    def hash_of_sequence(self,tup):
        MAX = 10e21
        sum = 1
        for i in tup:
            i = i+1 if i == 0 else i
            sum = sum * i
        sum = sum*len(tup)
        return sum % MAX
    
    def gen_fp(self):
        # Create a set of frequent pattern
        pattern_counts = []
        for key, value in self.relations.items():
            for relation in value:
                tmp = relation.copy()
                tmp.append((key, 1))
                pattern_counts.append([tuple([item[0] for item in tmp]), relation[-1][1]])
        # build dictionary
        all_relations = []
        for relation in pattern_counts:
            all_relations.append(relation[0])
        all_relations = frozenset(all_relations)
        # Create a dictionary using self.hash_of_sequence
        # { hash : (relation)}
        relation_dict = {}
        len_all_relations = len(all_relations)
        for relation in all_relations:
            relation_dict[self.hash_of_sequence(relation)] = relation
        # Create a dictionary of count using self.hash_of_sequence
        # {hash : counts}
        count_dict = {}
        for relation in pattern_counts:
            if hash_of_sequence(relation[0]) in count_dict:
                count_dict[hash_of_sequence(relation[0])] += relation[1]
            else :
                count_dict[hash_of_sequence(relation[0])] = relation[1]
        extract_frequent_pattern = []
        for key, value in count_dict.items():
            if value >= 10:
                extract_frequent_pattern.append((key, value))

        fp_n = {}
        fp_n_encoded = {}
        self.Max_fp = 0
        for fp in extract_frequent_pattern:
            original_pattern = relation_dict[fp[0]]
            if len(original_pattern) not in fp_n:
                fp_n[len(original_pattern)] = []
                fp_n_encoded[len(original_pattern)] = []
                if self.Max_fp < len(original_pattern):
                    self.Max_fp = len(original_pattern)
            t = fp_n[len(original_pattern)]
            t_encoded = fp_n_encoded[len(original_pattern)]
            t.append( (list(self.le.inverse_transform([p])[0] for p in original_pattern ), fp[1]) )
            t_encoded.append( (list(p for p in original_pattern ), fp[1])  )

        self.fp_n = fp_n
        self.fp_n_encoded = fp_n_encoded
        
    def support(self, com):
        com_s = set(com)
        count = 0
        for i, row in enumerate(self.encoded_data):
            if com_s.issubset(row) :
                count += 1
        return count/self.dshape[0]

    def gen_rule(self, min_conf):
        rules = []
        for m, m_c in self.fp_n_encoded[self.Max_fp-1]:
            for i in range(len(m)-1):
                for com in itertools.combinations(m, i+1):
                    sm = self.support(m)
                    scom = self.support(com)
                    if sm/scom >= min_conf:
                        rules.append({'rule': [com, tuple(set(m).difference(com))], 'confidence': sm/scom, 'support': sm})
        return rules
    def print_out_rules(self, rules):
        for dr in rules:
            print(f"{self.le.inverse_transform(dr['rule'][0])} => {self.le.inverse_transform(dr['rule'][1])}, with confidence {dr['confidence']} and support {dr['support']}")

In [262]:
data_df = pd.DataFrame([['a','c','d','f','g','i','m','p'], ['a','b','c','f','i','m','o'], ['b','f', 'h', 'j','o'],['b','c','k','s','p'], ['a','c','e','f','l','m','n','p'] ])

In [263]:
data_df = pd.read_csv(data_path, header=None)

In [264]:
data_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [265]:
fpe = FPEstimator(data_df[:1000])

In [266]:
#fpe.encoded_data

In [267]:
#fpe.l1label

In [268]:
fpe.build_tree(10, print_out=False)

In [269]:
fpe.root

In [270]:
# for d in fpe.ht:
#     cur = d['next']
#     while cur != None:
#         print(cur.key, cur.count)
#         cur = cur.next_instance
    

In [271]:
fpe.gen_list()

 99%|█████████▉| 79/80 [00:03<00:00, 21.18it/s]


In [272]:
f = open('fp-relation_test', 'w')
with tqdm(total = len(fpe.relations.keys())) as pbar:
    for key, value in fpe.relations.items():
        f.write(f"=================================\n")
        f.write(f"Find ({key} {fpe.le.inverse_transform([key])[0]})\n")
        for relation in value:
            for item in relation:
                f.write(f"({fpe.le.inverse_transform([item[0]])[0]} {item[1]}) ")
            f.write('\n')
        f.write('\n')
        pbar.update(1)

100%|██████████| 79/79 [00:00<00:00, 83.11it/s]


In [273]:
fpe.gen_fp()

In [274]:
fpe.fp_n

{2: [(['spaghetti', 'french wine'], 10),
  (['spaghetti', 'ham'], 13),
  (['chocolate', 'rice'], 10),
  (['mineral water', 'rice'], 11),
  (['mineral water', 'protein bar'], 10),
  (['french fries', 'strawberries'], 10),
  (['mineral water', 'fresh tuna'], 13),
  (['shrimp', 'pasta'], 21),
  (['burgers', 'almonds'], 10),
  (['milk', 'almonds'], 10),
  (['eggs', 'almonds'], 19),
  (['mineral water', 'almonds'], 39),
  (['green tea', 'cookies'], 17),
  (['milk', 'tomatoes'], 16),
  (['eggs', 'tomatoes'], 29),
  (['chocolate', 'yams'], 10),
  (['mineral water', 'yams'], 14),
  (['chocolate', 'pepper'], 12),
  (['spaghetti', 'pepper'], 11),
  (['eggs', 'pepper'], 20),
  (['mineral water', 'pepper'], 40),
  (['spaghetti', 'hot dogs'], 10),
  (['eggs', 'red wine'], 10),
  (['mineral water', 'red wine'], 14),
  (['milk', 'avocado'], 14),
  (['chocolate', 'avocado'], 10),
  (['mineral water', 'avocado'], 28),
  (['spaghetti', 'tomatoes'], 32),
  (['milk', 'fresh bread'], 10),
  (['french fries

In [282]:
rules = fpe.gen_rule(0.5)
fpe.print_out_rules(rules)

['green tea' 'tomatoes'] => ['frozen vegetables'], with confidence 0.5 and support 0.005
['milk' 'olive oil'] => ['mineral water'], with confidence 0.5789473684210527 and support 0.011
['spaghetti' 'whole wheat rice'] => ['chocolate'], with confidence 0.5 and support 0.007
['eggs' 'cooking oil'] => ['mineral water'], with confidence 0.7333333333333333 and support 0.011
['eggs' 'frozen smoothie'] => ['mineral water'], with confidence 0.5 and support 0.006
['chocolate' 'chicken'] => ['mineral water'], with confidence 0.5 and support 0.007
['chocolate' 'soup'] => ['milk'], with confidence 0.5 and support 0.01
['eggs' 'soup'] => ['milk'], with confidence 0.5 and support 0.009
['olive oil' 'soup'] => ['mineral water'], with confidence 0.7333333333333333 and support 0.011
['spaghetti' 'soup'] => ['mineral water'], with confidence 0.5882352941176471 and support 0.01
['chocolate' 'soup'] => ['mineral water'], with confidence 0.6 and support 0.012
['shrimp' 'cake'] => ['mineral water'], with co